In [123]:
import pandas as pd
from bs4 import BeautifulSoup as bts
import requests as req
import asyncio
from tqdm import tqdm
from typing import List, Tuple, Union
import re

In [3]:
with open('html/intro_naves.html','r',encoding='UTF-8') as file:
    contenido=file.read()
soup=bts(contenido,'html.parser')
href=[a.get('href').replace('..','https://space.skyrocket.de') for a in soup.find_all('table')[0].find_all('a') if int(a.text) > 1950 and int(a.text) < 2023]

In [4]:
def asincrono(funcion):
    ''' esto es un decorador que permite colocarlo encima de una funcion para poder ejecutar otras mientras se espera el resultado'''
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    
    return eventos

In [5]:
def DataFrame_table(url) -> pd.DataFrame:
    '''ulr contenido de la web que se va a transformar:
    DataFrame_table(url)
    ulr-> el contenido que va a leer y transformar
    return -> dataframe'''
    response=req.get(url)
    html_content=response.text
    soup=bts(html_content,'html.parser')

    table=soup.find('table',{'class':'data'})
    content_table=[content.text for content in table.find_all('td')]
    keys=[e.text for e in table.find('tr',{'class':'bigheadline'}) if e.text != '\n']
    values=[content_table[e:e+6] for e in range(0,len(content_table),6)]
    
    ref = soup.find('div', {'class': 'container'}).text.split('\n')
    corte1=soup.find('div',{'class': 'container'}).text.split('\n').index('Launch sites:')+1
    corte2=soup.find('div', {'class': 'container'}).text.split('\n').index('\t\r')-1
    colum_ref={e.split('=')[0].strip() : e.split('=')[-1].strip() for e in ref[corte1:corte2]}

    table_dic={key : [] for key in keys}


    for sublist in values:
        for a in range(6):
            table_dic[keys[a]].append(sublist[a])

    df=pd.DataFrame(table_dic)

    def fail(x) -> str:
        '''Simplifica los valores de las descripciones dadas como : fallo ,exito , tipo de vuelo y dudoso
        x -> es la str que entra para comprobar los parametros'''
        phrase=x.lower()

        if 'fail' in phrase:
            x='Failed'
            return x
        elif any(word in phrase for word in ['explo','wrong','damaged','abort','not deployed','error','Lower']):
            return 'Failed'
        elif any(word in phrase for word in ['1st','first flight']):
            return 'First flight'
        elif 'last flight' in phrase:
            return'Last flight'
        elif 'note' in phrase:
            return x
        else:
            return 'Pass'
    
    def complate(x) -> str:
        '''sustituye valores vacios como exitosos
        x -> es la str que se lee'''
        if x =='':
            x='Pass'
            return x
        else:
            return x
        
    def clean_df(df) -> pd.DataFrame:  
        ''' se encarga de ordenar y limpiar el df
        df -> dataframe que desea pasar por la limpieza'''
        df['Date']=df['Date'].apply(lambda x : x.replace('?','') if '?' in x else x)
        df['Date']=df['Date'].astype('datetime64[ns]')
        df['Remark']=df['Remark'].apply(complate)
        df['Payload(s)']=df['Payload(s)'].str.replace('\n',',')
        df['Remark']=df['Remark'].apply(fail)

        Succes=df['Remark']

        df_change=df.drop(columns='Remark')
        df_change['Site_inf']=df_change['Site'].apply(lambda x : x.split()[-1])
        df_change['Site']=df_change['Site'].apply(lambda x : x.split()[0])
        df_change['complate_site']=df_change['Site'].map(colum_ref)
        
        complete_df=pd.concat([df_change,Succes],axis=1)

        return complete_df
    
    df_clean=clean_df(df)

    return df_clean

@asincrono
def data(href) -> pd.DataFrame:
    '''a partir de una lista de url se crea un df :

    misiones=pd.DataFrame()

    for i in range(len(href)): -> lee la longitud de la lista para extraer df de manera individual 
    
        try:
            misiones=pd.concat([misiones,DataFrame_table(href[i])],axis=0) ->concatena el df con el principal para crear un dataframe unico 

        except Exception as e: -> en caso de querer sasber el fallo
            print(f'{e}:{i}')
            continue
        '''
    misiones=pd.DataFrame()
    for i in range(len(href)):
        try:
            misiones=pd.concat([misiones,DataFrame_table(href[i])],axis=0)
        except Exception as e:
            print(f'{e}:{i}')
            continue
    return misiones

In [6]:
def get_url(url)->list:
    '''entra una lista de urls/href que buscara las urls internas de lanzamientos
    devuelve una lista de urls de una pagina
    get_url() -> ejemplo : 
    1) insertar una unica url_get('https://aqui.web/pones/tu/url')
    return -> lista de urls que hay en una pagina web
    '''
    response=req.get(url)
    html_content=response.text
    soup=bts(html_content,'html.parser')

    table=soup.find('table',{'class':'data'})
    content_table=[content for content in table.find_all('td')]
    values=[content_table[e:e+6] for e in range(0,len(content_table),6)]
    filtered_values = [[td for td in sublist if any(a in td for a in td if a.name == 'a')] for sublist in values]
    launch_veicles=[]
    
    for a in filtered_values:
        url=a[-1].find('a').get('href').replace('..','https://space.skyrocket.de')
        launch_veicles.append(url)
    return launch_veicles

@asincrono
def url_launches(url) -> list:
    '''entra una lista de urls/href que buscara las urls internas de lanzamientos
    devuelve una lista dee urls de todas las paginas
    url_launches(url) -> se usa una lista de urls para podedr ejecutar la funcioin : get_url
    y lo guarda en una lista de valores unicos

    urls=[]  -> valores unicos 
    
    for i in url:
    
        list_url_page=get_url(i)
        for new_url in list_url_page:
            if new_url not in urls:
                urls.append(new_url)

    return urls -> devuelve lista de valores unicos'''
    urls=[]
    for i in url:
        list_url_page=get_url(i)
        for new_url in list_url_page:
            if new_url not in urls:
                urls.append(new_url)
    return urls

In [21]:
def rockets(url) -> pd.DataFrame:

    response=req.get(url)
    html_content=response.text
    soup=bts(html_content,'html.parser')

    try:
        '''hay dos tipos de tablas que me interesan porlo que si da error en una es la otra'''
        info=soup.find('table',{'id':'rocdata'}).find_all('tr')
        keys=[document.text for document in info[0] if document.text != '\n']

        try:
            '''hay dos tipos de diccionarios y para evitar errores pongo ambas cualidades'''
            values=[document.text for document in info[-1] if document.text != '\n']
            table_rocket={keys[e] : [values[e]] for e in range(len(keys))}

            df=pd.DataFrame(table_rocket)
            colum_launch=pd.DataFrame({'Launch Vehicle':[url.split('/')[-1].split('.')[0]]})
            final_df=pd.concat([colum_launch,df],axis=1)
        except:
            values=[e.strip() for e in [e.text for e in info[-1] if e.text!='\n'][0].split('/')]
            table_rocket={keys[e] : [values[e]] for e in range(len(keys))}

            df=pd.DataFrame(table_rocket)
            colum_launch=pd.DataFrame({'Launch Vehicle':[url.split('/')[-1].split('.')[0]]})
            final_df=pd.concat([colum_launch,df],axis=1)
        
    except:
        
        info=soup.find('table',{'id':'rocperf'}).find_all('tr')
        keys=[document.text for document in info[0] if document.text != '\n']
        values=[document.text for document in info[-1] if document.text != '\n']
        table_rocket={keys[e] : [values[e]] for e in range(len(keys))}

        df=pd.DataFrame(table_rocket)
        colum_launch=pd.DataFrame({'Launch Vehicle':[url.split('/')[-1].split('.')[0]]})
        final_df=pd.concat([colum_launch,df],axis=1)
        
    return final_df

def rockets_table(href) -> pd.DataFrame:

    final_df=pd.DataFrame()
    other_df=pd.DataFrame()
    for url in href:
        try:
            df=rockets(url)
            if 'LEO' not in list(df.columns):
                final_df=pd.concat([final_df,df],axis=0)
            else:
                other_df=pd.concat([other_df,df],axis=0)
        except:
            continue
    
    other_df=other_df[other_df.columns.to_list()[:3]]
    final_df=final_df[final_df.columns.to_list()[:-4]]
    
    return final_df,other_df

def lists_of_launches(href) -> tuple:

        '''
        lists_of_launches(href) => aqui se aplica la funcion: result1 = lists_of_launches('launch1', 'launch2', 'launch3')
        o launches = ['launch4', 'launch5', 'launch6']
        return => devuelve dos argumentos'''
        
        response = req.get(href)
        response_html=response.text
        soup = bts(response_html,'html.parser')
        table = soup.find('div',{'class':'llist'}).find_all('pre')
        clave_html = [e.find_all('a') for e in table if e.find_all('a')][0]
        keys = [e.text for e in clave_html]
        dicio_clave = {e.text:e.get('href').replace('..','https://space.skyrocket.de') for e in clave_html}

        lista_tablas = []
        lista_url_sat = [url for key,url in dicio_clave.items()]

        for content in table:
                for line in content.text.splitlines():
                        if any(key in line for key in keys):
                                lista_tablas.append(line)

        result=(lista_tablas,lista_url_sat)

        return result

def general_fun_of_lists(lists,fun) -> Union[list, Tuple]:
    '''
    general_fun_of_lists (lists,fun) => es una funcion que se encarga de iterar lista en las funciones que uno necesita
    litst => [una,lista,de,elementos,que,desea,trabajar]
    fun => la funcion que se va ha aplicar en cada elemento 
    return => depende de los resultados de la funcion que se use:

    1) da una lista unica => return solucion
    2) da una lista de dos resultados => return (solucion,solucion2)

    opcional: errores

    0) en caso de que haya errores esta funcion te enseña donde ha pasado, y el error:
       por eso en este caso puede ocurrir dos cosas:
       1)si la lista unica tiene errores, te devolvera una tupla donde el primero es la solucion y la segundd los errores => return (solucion, errores)
       2)si es una tupla de dos soluciones, te devuelve las soluciones mas el error => return (solucion,solucion2,errores)

    ejemplo:

    def encontrar_x() -> lista          fun de ejemplo

    general_fun_of_lists([una,lista,de,elementos,que,desea,trabajar],encontrar_x)
    '''
    sol = []
    sol_2 = []
    error = []

    for element in lists:

        try:
            result = fun(element)
            if len (result) == 1:
                sol.extend(result)
            elif len (result) == 2:
                sol.extend(result[0])
                sol_2.extend(result[-1])
        except Exception as e:
            error.append(f'{e}:{element}')

    if sol_2:
        if error:
            result = (sol,sol_2,error)
            return result
        else:
            result = (sol,sol_2)
            return result
    elif sol:
        if error:
            result = (sol,error)
            return result
        else:
            return sol


In [8]:
def visualizar_fallos(urls,fun):
    '''
    uso en caso de querer descubrir donde esta fallando la funcion:
    visualizar_fallos(urls,fun)
    urls -> urls que necesita leer la funcion para poner aprueba la peticion
    fun -> es la funcion que quieres testear para su funcionalidad
    return -> devuelve uns lista de errores
    '''
    errors=[]

    for i in range(len(urls)):
        try:
            fun(urls[i])
        except Exception as e:
            errors.append(f'{e}:{i}')
            
    
    return errors

In [9]:
help(visualizar_fallos)

Help on function visualizar_fallos in module __main__:

visualizar_fallos(urls, fun)
    uso en caso de querer descubrir donde esta fallando la funcion:
    visualizar_fallos(urls,fun)
    urls -> urls que necesita leer la funcion para poner aprueba la peticion
    fun -> es la funcion que quieres testear para su funcionalidad
    return -> devuelve uns lista de errores



In [11]:
misiones=data(href)
launches=url_launches(href)

In [12]:
misiones=misiones.result()
launches_list=launches.result()

In [13]:
rockes_1,rockes_2=rockets_table(launches_list)

In [17]:
rockes_1

,Launch Vehicle,Version,Strap-On,Stage 1,Stage 2,Stage 3
0,sputnik-1,Sputnik (8K71PS),"Blok-B,V,G,D / 4 × RD-107-8D74PS",Blok-A / RD-108-8D75PS,NaN,NaN
0,vanguard,Vanguard,NaN,X-405,AJ-10-37,GRC 133-KS-2800
0,juno-1,Juno-1,NaN,Redstone / A-7,11 × Baby Sergeant,3 × Baby Sergeant
0,sputnik-2,Sputnik (8A91),"Blok-B,V,G,D / 4 × RD-107-8D76",Blok-A / RD-108-8D77,NaN,NaN
0,pilot,Pilot (NOTS-EV1H),NaN,2 × 2 × HOTROC,X-241,NOTS-100
...,...,...,...,...,...,...
0,sslv,SSLV,NaN,S-85,S-7,S-4
0,falcon-9-heavy_b5_px,Falcon-Heavy (Block 5) (px),"2 × Stage 1 (str., reusable) / 9 × Merlin-1D",Stage 1 (str.) / 9 × Merlin-1D (upr.+),Stage 2 (str.) / Merlin-1D-Vac (upr.+),NaN
0,sls_bl1_icps,SLS (Block-1) iCPS,2 × RSRM-5,4 × RS-25D,iCPS / RL10B-2,NaN
0,jielong-3,Jielong-3 (Smart Dragon-3),NaN,?,?,?


In [18]:
rockes_2

,Launch Vehicle,Performance (kg),LEO
0,atlas-d_mercury,Atlas-D Mercury,1350
0,atlas-slv3,Atlas-SLV3,800
0,slv-3,SLV-3,40


In [22]:
resultado=general_fun_of_lists(launches_list,lists_of_launches)

In [33]:
lista_launches_trabajar,urls_sat,errores=resultado

In [39]:
clean=[[element for element in file.split('  ') if element != ''] for file in lista_launches_trabajar]

#RECONTRUCCION DE LOS DATOS 

In [159]:
lenght_of_info=set([len(e) for e in clean])
dicio_of_launches_info={e : [] for e in lenght_of_info}
for line in clean:
    dicio_of_launches_info[len(line)].append(line)
dicio_of_launches_info

dataframes=[e for e in lenght_of_info]
for element in lenght_of_info:
    dataframes[element-1]=pd.DataFrame(dicio_of_launches_info[element])

In [160]:
df_8=dataframes[8]
pattern = r'^\d{2}\.\d{2}\.\d{4}$'
df_8['date']=''

In [180]:
keys=[element.split('/')[-1].split('.')[0] for element in launches_list]

In [166]:
for column in df_8.columns:
    df_8[column]=df_8[column].str.strip()

In [167]:
for column in df_8.columns:
    df_8.loc[df_8[column].str.contains(pattern), 'date'] = df_8.loc[df_8[column].str.contains(pattern), column]

In [171]:
df_8['serial'] = df_8[2] + ' ' + df_8[3]

In [181]:
df_8['type']=''

In [183]:
for column in df_8.columns:
    df_8.loc[df_8[column].str.lower().isin(keys), 'type'] = df_8.loc[df_8[column].str.lower().isin(keys), column]

In [184]:
df_8

,0,1,2,3,4,5,6,7,8,date,serial,type
0,12,7,7,ST-7,Scout-X1,WI LA-3,*,19.10.1961,P 21,19.10.1961,7 ST-7,Scout-X1
1,15,9,1,ST-9,Scout-X2,WI LA-3,*,29.03.1962,P 21A,29.03.1962,1 ST-9,Scout-X2
2,26,19,4,116,Scout-X3,WI LA-3,*,22.05.1963,RFD 1,22.05.1963,4 116,Scout-X3
3,35,28,5,124R,Scout-X4,WI LA-3A,*,20.07.1964,SERT 1,20.07.1964,5 124R,Scout-X4
4,83,76,24,166CR,Scout-B,WI LA-3A,*,20.09.1971,BIC,20.09.1971,24 166CR,Scout-B
5,102,95,10,193C,Scout-D1,WI LA-3A,*,18.06.1976,GP A (Gravity Probe 1),18.06.1976,10 193C,Scout-D1
6,10,Ariane-5G,10,S,V-142 / 510,12.07.2001,Ko ELA-3,P,Artemis / BSat 2b,12.07.2001,10 S,Ariane-5G
7,14,Ariane-5ECA,1,L,V-157 / 517,11.12.2002,Ko ELA-3,F,Hotbird 7 / Stentor / MFD A / MFD B,11.12.2002,1 L,Ariane-5ECA
8,97,Ariane-5ECA,65,L,VA-241 / 5101,25.01.2018,Ko ELA-3,P,SES 14/GOLD / Al Yah 3,25.01.2018,65 L,Ariane-5ECA
9,1,1,LauncherOne,R2,25.05.2020,Mo RW12/30,B-747-400,F,Starshine 4 / Intern-Sat,25.05.2020,LauncherOne R2,LauncherOne
